In [1]:
# imports
from pathlib import Path
import rasterio
import re
import pandas as pd
import numpy as np
import json

In [ ]:
# Author: Ben Kraas
# Some help from copilot was used to write this code
# In a perfect world, this would need a thorough refactor to be less hardcoded

# Initialize the root path
root = Path("")

# terrible, terrible hardcoded dictionary to map the index to the year and month but it works and I will not change it
manual_match = {"1": "2020_1", "2": "2020_2", "3": "2020_3", "4": "2020_4",\
                "5": "2021_1", "6": "2021_2", "7": "2021_3", "8": "2021_4",\
                "9": "2022_1", "10": "2022_2", "11": "2022_3", "12": "2022_4",\
                "13": "2023_1", "14": "2023_2", "15": "2023_3", "16": "2023_4",\
                "17": "2024_1", "18": "2024_2", "19": "2024_3", "20": "2024_4"}
index_to_month_lut = {"1": "spring", "2": "summer", "3": "autumn", "4": "winter"}
year_mult_lut = {"2020": 1, "2021": 2, "2022": 3, "2023": 4, "2024": 5}

# Iterate over Sentinel 2 tifs in the data/source/sentinel2 directory and load all 13 bands from each single file
data_dict = {}
iterated_file_count = 0
had_warning = False

# Iterate over all tif files in the directory
for file in root.glob("data/source/sentinel-2-means-100m-tif-V2/*.tif"):
    print(f"Processing file: {file.name}")

    # Match the file name to the manual_match dictionary
    match = re.search(r"(\d{4})_(\d)", file.name)
    if match:
        year = match.group(1)
        index = match.group(2)
        findstring = f"{year}_{index}"
        if findstring in manual_match.values():
            final_name = f"S2_{year}_{index}_{index_to_month_lut[index]}.tif"
            print(f"  Found match: {final_name}")
            # Get the year and month from the file name
        else:
            print(f"  WARNING: No match found for {file.name}")
            continue

        raster_dict = {}

        # A wild formula appears!
        # position = int(index) * int(year_mult_lut[year]) # old and bad formula, probably ate brown M&Ms
        position = (int(index) - 1) + (int(year_mult_lut[year]) - 1) * 4 + 1
        print(f"  Opening file: {file.name} at position {position}")

        # Open the file with rasterio and prepare the bands
        with rasterio.open(file) as src:

            # Read all bands into a 2D numpy array
            bands = src.read()
            print(f"  File: {file.name}")
            print(f"  Number of bands: {src.count}")

            # get metadata outlining the band names
            band_names = src.descriptions

            # drop the eleventh band IF at index 11 of bandnames is "B10". This does not happen on our dataset.
            if band_names[10] == "B10":
                bandsbefore = bands[:10, :, :]
                bandsafter = bands[11:, :, :]
                bands = np.concatenate((bandsbefore, bandsafter), axis=0)
                had_warning = True
                print(f"  CAUTION: Dropping band 11 from {file.name} because it is B10")

            # Print the shape of the bands array
            print(f"  {file.name}: {bands.shape}")

            # Get the metadata of the raster
            meta = src.meta

            # assert number of bands matches the number of band names
            assert bands.shape[0] == len(band_names), f"  Number of bands {bands.shape[0]} does not match number of band names {len(band_names)}"

            # input the bands into the dictionary
            try:
                data_dict[position] = {
                    "bands": bands,
                    "band_names": band_names,
                    "meta": meta,
                    "file_name": file.name,
                }
            except KeyError as e:
                print(f"  WARNING: KeyError {e} for file {file.name}")
                had_warning = True
                continue
            iterated_file_count += 1

    else:
        print(f"  WARNING: No match found for {file.name}")

# sort the dictionary by the keys#
print(f"Number of files processed: {iterated_file_count}")
print(f"Had warning: {had_warning}")
print(f"Number of bands in dictionary: {len(data_dict)}")
sorted_data_dict = dict(sorted(data_dict.items()))

# print the file names in the sorted dictionary
for key, value in sorted_data_dict.items():
    print(f"Key: {key}, File name: {value['file_name']}")
print(f"Sorted data dictionary: {sorted_data_dict.keys()}")

Processing file: S2_2021_3_autumn.tif
  Found match: S2_2021_3_autumn.tif
  Opening file: S2_2021_3_autumn.tif at position 7
  File: S2_2021_3_autumn.tif
  Number of bands: 26
  S2_2021_3_autumn.tif: (26, 696, 820)
Processing file: S2_2020_2_summer.tif
  Found match: S2_2020_2_summer.tif
  Opening file: S2_2020_2_summer.tif at position 2
  File: S2_2020_2_summer.tif
  Number of bands: 26
  S2_2020_2_summer.tif: (26, 696, 820)
Processing file: S2_2024_4_winter.tif
  Found match: S2_2024_4_winter.tif
  Opening file: S2_2024_4_winter.tif at position 20
  File: S2_2024_4_winter.tif
  Number of bands: 26
  S2_2024_4_winter.tif: (26, 696, 820)
Processing file: S2_2022_2_summer.tif
  Found match: S2_2022_2_summer.tif
  Opening file: S2_2022_2_summer.tif at position 10
  File: S2_2022_2_summer.tif
  Number of bands: 26
  S2_2022_2_summer.tif: (26, 696, 820)
Processing file: S2_2022_3_autumn.tif
  Found match: S2_2022_3_autumn.tif
  Opening file: S2_2022_3_autumn.tif at position 11
  File: S2_2

In [ ]:
# get per band min and max values for the first entry in the dictionary
first_dict = data_dict[1]

# Get insights into the bands
for index, band_raster in enumerate(first_dict["bands"]):
    # get the min and max values for each band, ignore the nodata values
    band_min = np.nanmin(band_raster)
    band_max = np.nanmax(band_raster)
    print(f"Band {first_dict['band_names'][index]} min: {band_min}, max: {band_max}, type: {band_raster.dtype}, shape: {band_raster.shape}")

Band B1 min: 111.0, max: 4332.0, type: float64, shape: (696, 820)
Band B2 min: 148.58333333333334, max: 5319.6, type: float64, shape: (696, 820)
Band B3 min: 127.6, max: 5678.4, type: float64, shape: (696, 820)
Band B4 min: 100.25, max: 6202.2, type: float64, shape: (696, 820)
Band B5 min: 78.25, max: 6420.2, type: float64, shape: (696, 820)
Band B6 min: 59.333333333333336, max: 6344.6, type: float64, shape: (696, 820)
Band B7 min: 63.916666666666664, max: 6339.6, type: float64, shape: (696, 820)
Band B8 min: 65.4, max: 6630.4, type: float64, shape: (696, 820)
Band B8A min: 51.0, max: 6285.2, type: float64, shape: (696, 820)
Band B9 min: 30.6, max: 5080.846153846154, type: float64, shape: (696, 820)
Band B11 min: 42.0, max: 5360.0, type: float64, shape: (696, 820)
Band B12 min: 39.6, max: 5207.333333333333, type: float64, shape: (696, 820)
Band AOT min: 78.0, max: 149.4, type: float64, shape: (696, 820)
Band WVP min: 200.8, max: 943.8461538461538, type: float64, shape: (696, 820)
Band 

/tmp/ipykernel_344704/1699212721.py:6: RuntimeWarning: All-NaN slice encountered
  band_min = np.nanmin(band_raster)
/tmp/ipykernel_344704/1699212721.py:7: RuntimeWarning: All-NaN slice encountered
  band_max = np.nanmax(band_raster)


In [4]:
# Load wheat spectral map from csv (data/source/wheat_spectral_map.csv)

# Read the CSV skipping the units row (line 2, index 1)
spectral_map_lut = pd.read_csv("data/source/wheat_spectral_map.csv", skiprows=1)

# drop row where bandname(S2bandname) is B10
spectral_map_lut = spectral_map_lut.drop(spectral_map_lut[spectral_map_lut["bandname(S2bandname)"] == "B10"].index)

# Display the dataframe
print(spectral_map_lut.to_string(index=False))

bandname(S2bandname)  res(m)  wavelenth(nm)  reflectance(degree)                     description 
                  B1      60            443                0.024 Ultra Blue (Coastal and Aerosol)
                  B2      10            490                0.028                             Blue
                  B3      10            560                0.061                            Green
                  B4      10            665                0.031                              Red
                  B5      20            705                0.111 Visible and Near Infrared (VNIR)
                  B6      20            740                0.229 Visible and Near Infrared (VNIR)
                  B7      20            783                0.239 Visible and Near Infrared (VNIR)
                  B8      10            842                0.241 Visible and Near Infrared (VNIR)
                 B8A      20            865                0.242 Visible and Near Infrared (VNIR)
                  B9

In [ ]:
# At this point, a very broad assumption is necessary: We must assume that reflectance (the paper gave no unit) with Range (0-1) 
# can be mapped to the sentinel 2 bands (0-10000) by dividing those values by 10000
# This may be a very bad assumption, but it is the only one we have.

# Written by Ben Kraas
# This script takes the Sentinel 2 bands and the wheat spectral map and finds the pixels that match the spectral mapping
# for each band. It then outputs a new raster with the matching pixels.

# # # Configuration # # #

scaling_div_val = 10000
tolerance_fraction = 0.01
output_path = root / "data/output/spectral_match_strict_pxcount"
mode = "count" # count or distance_addition

# # # End of Configuration # # #


# Iterate over the sorted dictionary and process each image
for image_dict in data_dict.values():
    print(f"Processing image {image_dict['file_name']}")

    # get the bands from the dictionary
    bands = image_dict["bands"]
    band_names = image_dict["band_names"]
    meta = image_dict["meta"]
    
    # get spectral mapping list for the bands
    spectral_mapping_ls = []
    for band_index, band_name in enumerate(band_names):
        # skip if band index is higher than the number of rows in the spectral map lut
        if band_index >= len(spectral_map_lut):
            continue

        # get the row from the spectral map where the band name matches
        band_row = spectral_map_lut[spectral_map_lut["bandname(S2bandname)"] == band_name]

        # get the reflectance value from the reflectance(degree) column
        reflectance = band_row["reflectance(degree)"].values[0]
        spectral_mapping_ls.append(float(reflectance))
    print(f"  Spectral mapping list: {spectral_mapping_ls}")

    # Attempt to find pixels that are within the tolerance of the spectral mapping for each band
    # Scale the spectral mapping values to match the Sentinel 2 bands
    spectral_mapping_ls = [reflectance * scaling_div_val for reflectance in spectral_mapping_ls]
    print(f"  Scaled spectral mapping list: {spectral_mapping_ls}")

    # Iterate over each band and find pixels that match the spectral mapping
    # Initialize a list to store the indices of the pixels that match the spectral mapping
    output_raster = np.zeros(bands[0].shape, dtype=np.float64)
    print(f"  Output raster initialized with shape: {output_raster.shape}")

    # Find the matching pixels for each band
    print("  Finding matching pixels... Number of bands: ", len(bands))
    for band_index, band in enumerate(bands):
        # initialize the match raster with nan
        match_raster = np.full(band.shape, np.nan)

        # skip if band index is higher than the number of rows in the spectral map lut
        if band_index >= len(spectral_mapping_ls):
            continue

        # get the reflectance value from the spectral mapping list
        reflectance = round(spectral_mapping_ls[band_index], 3)

        # find pixels that are within the tolerance of the spectral mapping
        matching_pixels = np.abs(band - reflectance) <= (tolerance_fraction * reflectance)

        # give the match_raster the deviation from the reflectance value in percentage (0-1)
        match_raster[matching_pixels] = np.abs((band[matching_pixels] - reflectance) / reflectance) * 10
        print(f"    Match raster mean: {np.nanmean(match_raster)}, min: {np.nanmin(match_raster)}, max: {np.nanmax(match_raster)}")

        # Act depending on the mode
        match mode:
            case "distance_addition":
                # add each match raster deviation value on top of the output raster
                output_raster += np.nan_to_num(match_raster, nan=0.0)
            case "count":
                # add 1 to the output raster for each matching pixel
                output_raster[matching_pixels] += 1
            case _:
                print(f"  WARNING: Unknown mode {mode}. Skipping this band.")
                continue

        # Print the number of matching pixels for each band with padding
        print(f"    Band {band_names[band_index]:<4} chosen reflectance: {reflectance:<7} matching pixels: {np.sum(matching_pixels):<7}/"
              f"{band.size:<7} ({np.sum(matching_pixels) / band.size * 100:>6.2f}%) mean deviation: {np.nanmean(match_raster):<7.3f}")
    print(f"  Output raster shape: {output_raster.shape}, Output raster min: {np.nanmin(output_raster)}, max: {np.nanmax(output_raster)}")
    
    # get the metadata from the first band
    meta = image_dict["meta"]
    # update the metadata for the output raster
    meta.update({
        "dtype": "float64",
        "count": 1,
        "compress": "LZW",
        "driver": "GTiff",
        "nodata": np.nan,
    })

    # Save the output raster to a new file
    # make sure the output directory exists
    output_path.mkdir(parents=True, exist_ok=True)

    # get the output file name
    output_file_name = output_path / f"{image_dict['file_name']}"

    # remove the .tif extension from the file name
    output_file_name = output_file_name.with_name(output_file_name.stem + "_output.tif")

    # save the output raster to a new file, overwriting if it already exists
    with rasterio.open(output_file_name, "w", **meta) as dst:
        # write the pixel number raster to the output file
        dst.write(output_raster.astype("float64"), 1)
        print(f"  Saved output raster to {output_file_name}")


Processing image S2_2021_3_autumn.tif
  Spectral mapping list: [0.024, 0.028, 0.061, 0.031, 0.111, 0.229, 0.239, 0.241, 0.242, 0.237, 0.123, 0.069]
  Scaled spectral mapping list: [240.0, 280.0, 610.0, 310.0, 1110.0, 2290.0, 2390.0, 2410.0, 2420.0, 2370.0, 1230.0, 690.0000000000001]
  Output raster initialized with shape: (696, 820)
  Finding matching pixels... Number of bands:  26
    Match raster mean: 0.050615337021437265, min: 0.0, max: 0.09895833333333333
    Band B1   chosen reflectance: 240.0   matching pixels: 6413   /570720  (  1.12%) mean deviation: 0.051  
    Match raster mean: 0.04987286048313574, min: 0.0, max: 0.09970238095238163
    Band B2   chosen reflectance: 280.0   matching pixels: 6692   /570720  (  1.17%) mean deviation: 0.050  
    Match raster mean: 0.05028325358271377, min: 0.0, max: 0.09972677595628478
    Band B3   chosen reflectance: 610.0   matching pixels: 12309  /570720  (  2.16%) mean deviation: 0.050  
    Match raster mean: 0.04906269180931594, min: 0